In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

In [ ]:
train = pd.read_csv('/content/drive/My Drive/raw_data/train_v2.csv')
sample_submission = pd.read_csv('/content/drive/My Drive/raw_data/sample_submission_v2.csv')
transactions = pd.read_csv('/content/drive/My Drive/raw_data/transactions_v2.csv')
user_logs = pd.read_csv('/content/drive/My Drive/raw_data/user_logs_v2.csv')
members = pd.read_csv('/content/drive/My Drive/raw_data/members_v3.csv')


set the options so the output format can be displayed correctly

In [ ]:
pd.set_option('expand_frame_repr', True)
pd.set_option('display.max_rows', 30000000)
pd.set_option('display.max_columns', 100)

check the number of duplicate accounts in each table

In [ ]:
train.duplicated('msno').sum()
sample_submission.duplicated('msno').sum()
transactions.duplicated('msno').sum()
user_logs.duplicated('msno').sum()
members.duplicated('msno').sum()

0

returns the max value of numerical variables and membership_expire_date

returns the min value of transaction date

returns the mode of ordinal variable and dummy variables, if multiple values share the same frequency, keep the first one


In [ ]:
transactions_v2 = transactions.groupby('msno', as_index = False).agg(
                                            {'payment_method_id': lambda x:x.value_counts().index[0], 
                                             'payment_plan_days': 'max', 
                                             'plan_list_price': 'max',
                                             'actual_amount_paid': 'max', 
                                             'is_auto_renew': lambda x:x.value_counts().index[0], 
                                             'transaction_date': 'min', 
                                             'membership_expire_date': 'max',
                                             'is_cancel': lambda x:x.value_counts().index[0]})


returns the max value of date and number of unique songs

returns the sum of other variables

In [ ]:
user_logs_v2 = user_logs.groupby('msno', as_index = False).agg(
                                            {'date': 'max', 
                                             'num_25': 'sum', 
                                             'num_50': 'sum', 
                                             'num_75': 'sum',
                                             'num_985': 'sum', 
                                             'num_100': 'sum', 
                                             'num_unq': 'max', 
                                             'total_secs': 'sum'})

calculate the percentage of number of songs played within certain period

In [ ]:
user_logs_v2['percent_25'] = user_logs_v2['num_25']/(user_logs_v2['num_25']+user_logs_v2['num_50']+user_logs_v2['num_75']+user_logs_v2['num_985']+user_logs_v2['num_100'])
user_logs_v2['percent_50'] = user_logs_v2['num_50']/(user_logs_v2['num_25']+user_logs_v2['num_50']+user_logs_v2['num_75']+user_logs_v2['num_985']+user_logs_v2['num_100'])
user_logs_v2['percent_100'] = (user_logs_v2['num_985']+user_logs_v2['num_100'])/(user_logs_v2['num_25']+user_logs_v2['num_50']+user_logs_v2['num_75']+user_logs_v2['num_985']+user_logs_v2['num_100'])


drop useless variables

In [ ]:
user_logs_v3 = user_logs_v2.drop(columns = ['num_25', 'num_50', 'num_75', 'num_985', 'num_100'])

merge between different tables for modelling purpose

In [ ]:
dataset_train = train.merge(members, on = 'msno', how = 'left').merge(transactions_v2, on = 'msno', how = 'left').merge(user_logs_v3, on = 'msno', how = 'left')
dataset_train.dtypes

msno                       object
is_churn                    int64
city                      float64
bd                        float64
gender                     object
registered_via            float64
registration_init_time    float64
payment_method_id         float64
payment_plan_days         float64
plan_list_price           float64
actual_amount_paid        float64
is_auto_renew             float64
transaction_date          float64
membership_expire_date    float64
is_cancel                 float64
date                      float64
num_unq                   float64
total_secs                float64
percent_25                float64
percent_50                float64
percent_100               float64
dtype: object

date in csv will be recognized as float in python
this value needs to be converted back to date

In [ ]:
dataset_train['registration_init_time'] = pd.to_datetime(dataset_train['registration_init_time'], format = '%Y%m%d')
dataset_train['transaction_date'] = pd.to_datetime(dataset_train['transaction_date'], format = '%Y%m%d')
dataset_train['membership_expire_date'] = pd.to_datetime(dataset_train['membership_expire_date'], format = '%Y%m%d')
dataset_train['date'] = pd.to_datetime(dataset_train['date'], format = '%Y%m%d')


check the maximum of datetime value

In [ ]:
dataset_train.select_dtypes(include = ['datetime64[ns]']).max()

registration_init_time   2017-04-24
transaction_date         2017-03-31
membership_expire_date   2023-08-17
date                     2017-03-31
dtype: datetime64[ns]

create new day columns for modelling purpose

In [ ]:
dataset_train['registration_day'] = (dataset_train['membership_expire_date'].max() - dataset_train['registration_init_time']).astype('timedelta64[D]')
dataset_train['transaction_day'] = (dataset_train['membership_expire_date'].max() - dataset_train['transaction_date']).astype('timedelta64[D]')
dataset_train['membership_expire_day'] = (dataset_train['membership_expire_date'].max() - dataset_train['membership_expire_date']).astype('timedelta64[D]')
dataset_train['last_play_day'] = (dataset_train['membership_expire_date'].max() - dataset_train['date']).astype('timedelta64[D]')

check the distribution of age 

In [ ]:
dataset_train['bd'].value_counts()

 0.0       473729
 27.0       22786
 26.0       21829
 25.0       20651
 24.0       20296
 28.0       20244
 29.0       19533
 23.0       19238
 22.0       18035
 30.0       17175
 21.0       15633
 31.0       15123
 32.0       14406
 20.0       13678
 33.0       13229
 34.0       12154
 35.0       10959
 36.0        9985
 19.0        9861
 37.0        8858
 18.0        8207
 38.0        7679
 39.0        6651
 17.0        5913
 40.0        5663
 41.0        5221
 42.0        4305
 43.0        3723
 44.0        3266
 46.0        3015
 45.0        3010
 47.0        2889
 16.0        2544
 48.0        2539
 49.0        2320
 50.0        2027
 51.0        1970
 52.0        1748
 53.0        1445
 54.0        1159
 55.0        1021
 15.0         848
 56.0         844
 57.0         736
 58.0         541
 59.0         510
 60.0         455
 61.0         390
 62.0         309
 14.0         282
 63.0         258
 64.0         193
 65.0         171
 66.0         147
 67.0         140
 68.0     

remove gender and age since missing value or incorrect value is over 50%

In [ ]:
dataset_train_v2 = dataset_train.drop(columns = ['msno', 'gender', 'bd', 'registration_init_time', 'transaction_date', 'membership_expire_date', 'date'])
dataset_train_v2.dtypes

is_churn                   int64
city                     float64
registered_via           float64
payment_method_id        float64
payment_plan_days        float64
plan_list_price          float64
actual_amount_paid       float64
is_auto_renew            float64
is_cancel                float64
num_unq                  float64
total_secs               float64
percent_25               float64
percent_50               float64
percent_100              float64
registration_day         float64
transaction_day          float64
membership_expire_day    float64
last_play_day            float64
dtype: object

check the number of missing values in each column

In [ ]:
dataset_train_v2.isna().sum()

is_churn                      0
city                     109993
registered_via           109993
payment_method_id         37382
payment_plan_days         37382
plan_list_price           37382
actual_amount_paid        37382
is_auto_renew             37382
is_cancel                 37382
num_unq                  216409
total_secs               216409
percent_25               216409
percent_50               216409
percent_100              216409
registration_day         109993
transaction_day           37382
membership_expire_day     37382
last_play_day            216409
dtype: int64

Handle missing value of part of numeric columns by using mode

In [ ]:
def replacemode(i):
    dataset_train_v2[i] = dataset_train_v2[i].fillna(dataset_train_v2[i].value_counts().index[0])
    return 

replacemode('city')
replacemode('registered_via')
replacemode('payment_method_id')
replacemode('payment_plan_days')
replacemode('is_auto_renew')
replacemode('is_cancel')

Handle missing value of part of numeric columns by using mean

In [ ]:
from sklearn.impute import SimpleImputer
mean_imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
def replacemean(i):
    dataset_train_v2[i] = mean_imputer.fit_transform(dataset_train_v2[[i]])
    return 

replacemean('plan_list_price')
replacemean('actual_amount_paid')
replacemean('num_unq')
replacemean('total_secs')
replacemean('percent_25')
replacemean('percent_50')
replacemean('percent_100')
replacemean('registration_day')
replacemean('transaction_day')
replacemean('membership_expire_day')
replacemean('last_play_day')

Handle outliers by using capping

In [ ]:
def replaceoutlier(i):
    mean, std = np.mean(dataset_train_v2[i]), np.std(dataset_train_v2[i])
    cut_off = std*3
    lower, upper = mean - cut_off, mean + cut_off
    dataset_train_v2[i][dataset_train_v2[i] < lower] = lower
    dataset_train_v2[i][dataset_train_v2[i] > upper] = upper
    return

replaceoutlier('plan_list_price')
replaceoutlier('actual_amount_paid')
replaceoutlier('num_unq')
replaceoutlier('total_secs')
replaceoutlier('percent_25')
replaceoutlier('percent_50')
replaceoutlier('percent_100')
replaceoutlier('registration_day')
replaceoutlier('transaction_day')
replaceoutlier('membership_expire_day')
replaceoutlier('last_play_day')

dataset_train_v2.dtypes
dataset_train_v2.describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,is_churn,city,registered_via,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,num_unq,total_secs,percent_25,percent_50,percent_100,registration_day,transaction_day,membership_expire_day,last_play_day
count,970960.000000,970960.000000,970960.000000,970960.000000,970960.000000,970960.000000,970960.000000,970960.000000,970960.000000,970960.000000,970960.000000,970960.000000,970960.000000,970960.000000,970960.000000,970960.000000,970960.000000,970960.000000
mean,0.089942,5.377985,6.904894,38.980580,33.870175,137.312482,137.144432,0.915621,0.018600,64.235962,133525.936898,0.183335,0.046258,0.738445,3622.760136,2348.038834,2307.555337,2332.506040
std,0.286099,6.264004,1.825053,3.735683,33.145373,59.457891,59.592053,0.277956,0.135108,39.653047,128387.816988,0.124044,0.037513,0.162003,1027.431145,18.778962,24.720329,4.005140
min,0.000000,1.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.035000,0.000000,0.000000,0.226005,2306.000000,2330.000000,2158.670752,2330.000000
25%,0.000000,1.000000,7.000000,38.000000,30.000000,99.000000,99.000000,1.000000,0.000000,37.000000,43109.197250,0.095390,0.021978,0.683026,2810.000000,2338.000000,2303.000000,2330.000000
50%,0.000000,1.000000,7.000000,41.000000,30.000000,149.000000,149.000000,1.000000,0.000000,65.123435,126896.534500,0.185996,0.046786,0.736224,3551.000000,2347.000000,2311.000000,2331.000000
75%,0.000000,11.000000,7.000000,41.000000,30.000000,149.000000,149.000000,1.000000,0.000000,76.000000,140794.615500,0.219114,0.050445,0.852273,4062.000000,2355.000000,2320.000000,2332.705707
max,1.000000,22.000000,13.000000,41.000000,450.000000,564.946789,564.957389,1.000000,1.000000,198.663728,630426.734475,0.591411,0.202598,1.000000,6725.205993,2507.613742,2361.000000,2347.258221


convert categorical variables into string

In [ ]:
dataset_train_v2.iloc[:, 1:4] = dataset_train_v2.iloc[:, 1:4].astype(str)

replace discrete features with historical churn rate

In [ ]:
city_mean = pd.DataFrame(dataset_train_v2.groupby('city')['is_churn'].mean().reset_index())
city_mean.rename(columns = {'is_churn': 'city_mean'}, inplace=True)
register_mean = pd.DataFrame(dataset_train_v2.groupby('registered_via')['is_churn'].mean().reset_index())
register_mean.rename(columns = {'is_churn': 'register_mean'}, inplace=True)
payment_mean = pd.DataFrame(dataset_train_v2.groupby('payment_method_id')['is_churn'].mean().reset_index())
payment_mean.rename(columns = {'is_churn': 'payment_mean'}, inplace=True)

dataset_train_v3 = dataset_train_v2.merge(city_mean, on = 'city', how = 'left').merge(register_mean, on = 'registered_via', how = 'left').merge(payment_mean, on = 'payment_method_id', how = 'left')
dataset_train_v3 = dataset_train_v3.drop(columns = ['city', 'registered_via', 'payment_method_id'])


Feature Scaling for modelling purpose by using both min-max-scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X = dataset_train_v3.drop(columns = ['is_churn'])
Y = dataset_train_v3['is_churn']
nm_X = pd.DataFrame(MinMaxScaler().fit_transform(X))
nm_X.columns = X.columns.values
nm_X.index = X.index.values


Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif
nm_col = ['is_auto_renew', 'is_cancel']
nm_X_v2 = nm_X.drop(columns = nm_col)
nm_X_v3 = pd.DataFrame(nm_X, columns = nm_col)
nm_X_v4 = pd.DataFrame(SelectKBest(score_func=chi2, k='all').fit(nm_X_v3, Y).pvalues_ <= 0.05, columns = ['importance'])
nm_X_v4.index = nm_X_v3.columns.values
nm_X_v5 = pd.DataFrame(SelectKBest(score_func=f_classif, k='all').fit(nm_X_v2, Y).pvalues_ <= 0.05, columns = ['importance'])
nm_X_v5.index = nm_X_v2.columns.values
nm_X_v6 = pd.concat([nm_X_v4, nm_X_v5])
nm_selected = list(pd.Series(nm_X_v6[nm_X_v6['importance'] == 1].index.values))
nm_X_v7 = pd.DataFrame(nm_X, columns = nm_selected)


Dimension Reduction

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit_transform(nm_X_v7)
np.cumsum(pca.explained_variance_ratio_)
nm_X_v8 = pd.DataFrame(PCA(n_components=10).fit_transform(nm_X_v7),columns = [""])


In [ ]:
nm_X_v8.head()

,0,1,2,3,4,5,6,7,8,9
0,0.628673,-0.062653,-0.182833,0.264305,0.238114,0.053275,0.065127,-0.032926,0.289713,0.010577
1,0.708583,-0.451231,0.731810,-0.325409,0.029277,0.157258,-0.413314,-0.081948,-0.378832,0.067124
2,0.914509,0.012337,1.074347,0.237393,-0.278264,0.190097,0.168211,0.044930,0.093576,-0.102084
3,-0.323182,0.114473,0.120561,-0.025698,-0.185904,0.195808,0.614533,-0.071374,-0.036722,0.938720
4,0.225305,0.239037,-0.357108,-0.140872,-0.498801,0.220262,0.406497,-0.080835,-0.100077,0.968652


In [ ]:
nm_X_v9 = pd.DataFrame(pca.components_,columns=nm_X_v7.columns)
nm_X_v9.head()

,is_auto_renew,is_cancel,payment_plan_days,plan_list_price,actual_amount_paid,num_unq,total_secs,percent_50,percent_100,transaction_day,membership_expire_day,last_play_day,city_mean,register_mean,payment_mean
0,-0.276471,-0.002692,0.038861,0.123910,0.123469,0.104244,0.102603,-0.022555,0.044343,0.011941,-0.078677,-0.079763,0.732112,0.562388,0.051138
1,-0.234743,-0.041709,0.034023,0.038314,0.038581,0.442105,0.513503,-0.254558,0.335498,0.000629,-0.038354,-0.473963,-0.243562,-0.109232,0.071938
2,-0.737074,-0.017458,0.100873,0.143134,0.143773,-0.097180,-0.157652,0.219386,-0.277573,-0.002621,-0.109569,0.038507,-0.405567,0.143382,0.216957
3,0.156743,-0.042159,-0.033980,-0.051002,-0.051117,0.301152,0.119460,0.530279,-0.599431,-0.005108,0.034711,-0.431794,0.125500,-0.121629,-0.059900
4,0.168440,0.082349,-0.117216,-0.119741,-0.120783,0.318604,0.269716,0.071402,-0.101965,-0.040815,0.194723,0.368719,-0.373736,0.627642,-0.152579


In [ ]:
pca.components_

array([[-2.76470883e-01, -2.69173914e-03,  3.88612043e-02,
         1.23910328e-01,  1.23468970e-01,  1.04243814e-01,
         1.02602569e-01, -2.25552256e-02,  4.43425151e-02,
         1.19409344e-02, -7.86765362e-02, -7.97631565e-02,
         7.32112492e-01,  5.62388181e-01,  5.11379404e-02],
       [-2.34742908e-01, -4.17094977e-02,  3.40230552e-02,
         3.83144789e-02,  3.85805547e-02,  4.42104552e-01,
         5.13502905e-01, -2.54557807e-01,  3.35498301e-01,
         6.28540774e-04, -3.83539197e-02, -4.73963145e-01,
        -2.43561735e-01, -1.09232303e-01,  7.19382162e-02],
       [-7.37074216e-01, -1.74576317e-02,  1.00872713e-01,
         1.43134268e-01,  1.43772654e-01, -9.71801114e-02,
        -1.57651706e-01,  2.19386231e-01, -2.77573011e-01,
        -2.62104673e-03, -1.09569004e-01,  3.85066458e-02,
        -4.05566817e-01,  1.43382483e-01,  2.16957198e-01],
       [ 1.56743069e-01, -4.21591843e-02, -3.39803395e-02,
        -5.10016683e-02, -5.11165237e-02,  3.01152077

saving the processed_data

In [ ]:
data = pd.concat([pd.DataFrame(nm_X_v8),pd.DataFrame(Y)], axis=1, join='inner')

data.to_csv("/content/drive/My Drive/processed_data/final.csv",index=False)


making demos

In [ ]:
data.sample(n=100).to_csv("/content/drive/My Drive/processed_data/demo.csv",index=False)

In [ ]:
data.head()

NameError: ignored